In [9]:
from gamec import *
from display import GomokuUI
from copy import deepcopy
import pygame
import numpy as np
import numpy as np
import copy

# 定义无穷大
INF = 999999999

def advantage_f(l3, l4, my_player):
    """
    局势评估函数
    注意：建议大幅提高 l4 的权重，因为活四通常意味着必胜/必防
    """
    opponent = 3 - my_player
    score = 0
    # 调整权重：活4极其重要，给极高分
    score += (l4.get(my_player, 0) - l4.get(opponent, 0)) * 20
    score += (l3.get(my_player, 0) - l3.get(opponent, 0)) * 1
    return score

def advantage_vector(l3, l4, my_player):
    opponent = 3 - my_player
    vec = np.array([
        l4.get(my_player, 0),l4.get(opponent, 0),
        l3.get(my_player, 0),l3.get(opponent, 0)
    ])
    # 其中，在maximizing中，应该关注我方的L4 L3，即vec[0], vec[2]
    # 在minimizing中，应该关注我方的L4 L3 以及对方的L4，即vec[0], vec[2], vec[1]
    return vec

def basic_ai_move(game:GomokuCore, my_player, depth=3):
    """
    AI 入口函数
    :param game: 当前的 GomokuCore 实例
    :param my_player: AI 的棋子颜色 (1 或 2)
    :param depth: 搜索深度
    :return: (row, col) 最佳落子点
    """
    # 1. 获取候选点 (一定要用之前的 Bounding Box 逻辑减少搜索范围)
    candidates = game.recommand_positions()

    best_score = -INF
    best_move = candidates[0] # 默认一个合法值

    # Root 层搜索 (Maximizing Layer)
    adv_saved = advantage_vector(game.l3_count, game.l4_count, my_player)

    best_kaisha = -1
    best_rc = (None, None)
    for r, c in candidates:
        # --- 核心策略：深拷贝产生新宇宙 ---
        next_game = copy.deepcopy(game)
        
        success = next_game.place_stone(r, c)
        
        if success:
            # 如果这步直接赢了，直接返回，不需要再算了
            if next_game.winner == my_player:
                return (r, c)

            # 进入递归 (下一层轮到对手，所以是 Minimize)
            # 注意：传入 next_game (副本)
            
            chaa = advantage_vector(next_game.l3_count, next_game.l4_count, my_player) - adv_saved
            judgement = chaa[0] !=0 or chaa[2] !=0
            if judgement or next_game.game_over:
                print(r,c)
                res, dpt = kill( next_game, 5, -INF, INF, False, my_player)
                if res == INF and dpt > best_kaisha:
                    best_kaisha = dpt
                    best_rc = (r, c)
                    print(f"发现杀招 at depth {dpt} : ({r}, {c})")
    if best_kaisha != -1:
        return best_rc
    print("没杀掉")

    for r, c in candidates:
        # --- 核心策略：深拷贝产生新宇宙 ---
        next_game = copy.deepcopy(game)
        success = next_game.place_stone(r, c)
        if success:
            # 如果这步直接赢了，直接返回，不需要再算了
            if next_game.winner == my_player:
                return (r, c)
            # 进入递归 (下一层轮到对手，所以是 Minimize)
            # 注意：传入 next_game (副本)
            score = minimax(next_game, depth - 1, -INF, INF, False, my_player)
            if score > best_score:
                best_score = score
                best_move = (r, c)
                
    return best_move

def kill(game:GomokuCore, depth, alpha, beta, is_maximizing, my_player):
    """
    带排序剪枝(Beam Search)的 Minimax
    """
    # 1. 检查游戏结束
    if game.game_over:
        if game.winner == my_player:
            return INF, depth
        elif game.winner == (3 - my_player):
            return -INF, depth
        else:
            return 0, depth
    adv_saved = advantage_vector(game.l3_count, game.l4_count, my_player)
    # 2. 达到深度限制 (Leaf Node)
    if depth == 0:
        return advantage_f(game.l3_count, game.l4_count, my_player), depth
    # 3. 生成并评估所有候选状态 (这是本层的核心开销)
    candidates = game.recommand_positions()
    scored_moves = [] # 格式: (score, next_game_instance)


    # score += (l4.get(my_player, 0) - l4.get(opponent, 0)) * 20
    # score += (l3.get(my_player, 0) - l3.get(opponent, 0)) * 1



    for r, c in candidates:
        # 这里的 deepcopy 是必须的，为了计算该状态的得分
        next_game = copy.deepcopy(game)
        success = next_game.place_stone(r, c)
        
        if success:
            # 如果这一步直接导致游戏结束，赋予极值，确保它会被排在第一位
            if next_game.game_over:
                current_score = INF if next_game.winner == my_player else -INF
            else:
                # 计算启发式分数 (Heuristic Score)
                current_score = advantage_f(next_game.l3_count, next_game.l4_count, my_player)
            current_vec = advantage_vector(next_game.l3_count, next_game.l4_count, my_player)
            chaa = current_vec - adv_saved # chaa-差
            judgement1 = (chaa[0] !=0 or chaa[2] !=0 ) and is_maximizing
            judgement2 = (chaa[2]!=0 or chaa[1] !=0 or chaa[0] !=0 )and (not is_maximizing)
            if judgement1 or judgement2 or next_game.game_over:
                scored_moves.append((current_score, next_game))
    print(f"len:{len(scored_moves)}, depth:{depth}")           
    # 4. 排序与截断 (Sorting & Pruning)
    # 如果没有合法走法 (比如平局填满)，直接返回平局分
    if not scored_moves and is_maximizing:
        return 0, depth
    elif not scored_moves and (not is_maximizing):
        scored_moves.append((current_score, next_game))
    max_eval = -INF
    dpt_best = -1
    if is_maximizing:
        # Max层：希望分数越高越好，所以降序排列 (Reverse=True)
        scored_moves.sort(key=lambda x: x[0], reverse=True)
        for _, next_game_state in scored_moves:
            # 递归时不需要再 deepcopy 了，因为 scored_moves 里存的已经是独立的副本
            eval_score, dpt = kill(next_game_state, depth - 1, alpha, beta, False, my_player)
            dpt_best = max(dpt_best, dpt)
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, dpt_best
    else:
        # Min层：对手希望分数越低越好(对我越不利)，所以升序排列 (Reverse=False)
        scored_moves.sort(key=lambda x: x[0], reverse=False)
        min_eval = INF
        for _, next_game_state in scored_moves:
            eval_score, dpt = kill(next_game_state, depth - 1, alpha, beta, True, my_player)
            dpt_best = max(dpt_best, dpt)
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, dpt_best

def minimax(game:GomokuCore, depth, alpha, beta, is_maximizing, my_player, top_k=3):
    """
    带排序剪枝(Beam Search)的 Minimax
    """
    # 1. 检查游戏结束
    if game.game_over:
        if game.winner == my_player:
            return INF
        elif game.winner == (3 - my_player):
            return -INF
        else:
            return 0

    # 2. 达到深度限制 (Leaf Node)
    if depth == 0:
        return advantage_f(game.l3_count, game.l4_count, my_player)

    # 3. 生成并评估所有候选状态 (这是本层的核心开销)
    candidates = game.recommand_positions()
    scored_moves = [] # 格式: (score, next_game_instance)

    for r, c in candidates:
        # 这里的 deepcopy 是必须的，为了计算该状态的得分
        next_game = copy.deepcopy(game)
        success = next_game.place_stone(r, c)
        
        if success:
            # 如果这一步直接导致游戏结束，赋予极值，确保它会被排在第一位
            if next_game.game_over:
                current_score = INF if next_game.winner == my_player else -INF
            else:
                # 计算启发式分数 (Heuristic Score)
                current_score = advantage_f(next_game.l3_count, next_game.l4_count, my_player)
            
            scored_moves.append((current_score, next_game))

    # 4. 排序与截断 (Sorting & Pruning)
    # 如果没有合法走法 (比如平局填满)，直接返回平局分
    if not scored_moves:
        return 0

    if is_maximizing:
        # Max层：希望分数越高越好，所以降序排列 (Reverse=True)
        scored_moves.sort(key=lambda x: x[0], reverse=True)
        
        # 只取前 top_k
        best_moves = scored_moves[:top_k]
        
        max_eval = -INF
        for _, next_game_state in best_moves:
            # 递归时不需要再 deepcopy 了，因为 scored_moves 里存的已经是独立的副本
            eval_score = minimax(next_game_state, depth - 1, alpha, beta, False, my_player, top_k)
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval

    else:
        # Min层：对手希望分数越低越好(对我越不利)，所以升序排列 (Reverse=False)
        scored_moves.sort(key=lambda x: x[0], reverse=False)
        
        # 只取前 top_k
        best_moves = scored_moves[:top_k]
        
        min_eval = INF
        for _, next_game_state in best_moves:
            eval_score = minimax(next_game_state, depth - 1, alpha, beta, True, my_player, top_k)
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval

In [10]:

def main():
    # 1. 实例化逻辑层和显示层
    game = GomokuCore(board_size=15)
    ui = GomokuUI(board_size=15, cell_size=40)
    player = 3- game.current_player
    clock = pygame.time.Clock()
    running = True

    while running:
        # --- 事件处理 (Input) ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            elif event.type == pygame.MOUSEBUTTONDOWN and game.current_player == player:
                if event.button == 1: # 左键点击
                    if game.game_over:
                        game.reset()
                    else:
                        # UI 负责将点击转换成坐标
                        row, col = ui.convert_mouse_to_grid(event.pos)
                        # Logic 负责判断是否合法并更新数据
                        game.place_stone(row, col)
        
        # --- 渲染循环 (Output) ---
        # 显示层只需要当前的数据状态
        ui.draw(
            board_array=game.get_board(), 
            current_player=game.current_player,
            game_over=game.game_over,
            winner=game.winner,
            l3_count=game.l3_count,
            l4_count=game.l4_count,
            last_move=game.last_move
        )
        if player != game.current_player:
            row,col = basic_ai_move(game, my_player=game.current_player, depth=5)
            print("AI落子:",row,col)
            game.place_stone(row, col)
        clock.tick(30) # 限制30帧，节省资源

    pygame.quit()

if __name__ == "__main__":
    main()

没杀掉
AI落子: 7 7
没杀掉
AI落子: 6 6
8 8
len:2, depth:5
len:2, depth:4
len:1, depth:3
len:0, depth:2
len:1, depth:3
len:0, depth:2
len:2, depth:4
len:1, depth:3
len:0, depth:2
5 5
len:2, depth:5
len:2, depth:4
len:1, depth:3
len:0, depth:2
len:1, depth:3
len:0, depth:2
len:2, depth:4
len:1, depth:3
len:0, depth:2
4 4
len:2, depth:5
len:0, depth:4
len:2, depth:4
len:1, depth:3
len:0, depth:2
9 9
len:2, depth:5
len:0, depth:4
len:2, depth:4
len:1, depth:3
len:0, depth:2
没杀掉
AI落子: 8 5
5 5
len:2, depth:5
len:4, depth:4
len:1, depth:3
len:2, depth:2
len:2, depth:1
len:2, depth:1
len:1, depth:3
len:2, depth:2
len:2, depth:1
len:2, depth:1
len:2, depth:3
len:6, depth:2
len:1, depth:1
len:1, depth:1
len:2, depth:1
len:2, depth:1
len:2, depth:1
len:2, depth:1
len:2, depth:3
len:4, depth:2
len:5, depth:1
len:5, depth:1
len:6, depth:1
len:5, depth:1
len:4, depth:4
len:1, depth:3
len:4, depth:2
len:2, depth:1
len:2, depth:1
8 8
len:2, depth:5
len:4, depth:4
len:1, depth:3
len:2, depth:2
len:2, depth:1
len: